## This file helps to quickly write code and test it and alter on we will put in respective files as per modular coding

In [49]:

import langchain
print(langchain.__version__)
import sys
print(sys.executable)


1.2.0
/Users/amitpal/Desktop/GitRepos/RAG_ML_chatbot/venv/bin/python


In [50]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from dotenv import load_dotenv
import os
load_dotenv()

True

In [51]:
def file_loader(path):
    loader=DirectoryLoader(
        path,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )    
    documents=loader.load()
    return documents

In [52]:
extracted_docs=file_loader("data/")

In [53]:
def chunking_data(split_documents):
    text_splitter=RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=50
    )
    chunk_data=text_splitter.split_documents(split_documents)
    return chunk_data

In [54]:
chunk_data=chunking_data(extracted_docs)
print(len(chunk_data))

1680


In [55]:
def get_embedding():
    embedding=OpenAIEmbeddings(model="text-embedding-3-small")
    return embedding

In [56]:
embedding=get_embedding()

In [57]:
vector_store=FAISS.from_documents(documents=chunk_data,embedding=embedding)
vector_store

In [58]:
retriever=vector_store.as_retriever(search_type="similarity",search_kwargs={"k":3})
#response=retriever.invoke("what is supervised machine learning")
#response

In [59]:
model=ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.5
)

In [69]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate

system_prompt=(
    "Use the following context to answer the question"
    "If you don't find any context related to query just reply as no"
    "Don't give hallucinated answer of this"
    "Answer it in concise way"
    "\n\n"
    "Context:"
    "{context}"
   
)
prompt=ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("user","{question}")
    ]
)

rag_chain=(
    {
        "context":retriever,
        "question":RunnablePassthrough()
    }|prompt|model
)

response=rag_chain.invoke("What is supervised learning?")
print(response.content)



Supervised learning is a type of machine learning where a model is trained on input/output pairs from a training set to predict outcomes for new, unseen data. It typically requires human effort to create the training set.
